### Import Packages

In [1]:
import glob
import pandas as pd
import plotly.express as px

In [2]:
DATA_DIR = '../test_results/'

In [3]:
files = [file for file in glob.glob(DATA_DIR + '*.json')]

In [4]:
files

['../test_results\\charembed_bilstm.json',
 '../test_results\\charembed_bilstm_vec.json',
 '../test_results\\charembed_maxout.json',
 '../test_results\\charembed_maxout_vec.json',
 '../test_results\\charembed_mish.json',
 '../test_results\\charembed_mish_vec.json',
 '../test_results\\multihash_bilstm.json',
 '../test_results\\multihash_bilstm_vec.json',
 '../test_results\\multihash_maxout.json',
 '../test_results\\multihash_maxout_vec.json',
 '../test_results\\multihash_mish.json',
 '../test_results\\multihash_mish_vec.json']

In [74]:
def make_df(file):
    # Read .json
    df = pd.read_json(file, orient='index')
    # Transpose to get in right format
    df = df.transpose()
    # Add column with name of the model
    df['model'] = str(file).lstrip('../test_results\\').rstrip('.json')
    # Get scores for each label category as individual columns
    ent_scores = pd.json_normalize(df['ents_per_type'])
    # Concatenate columns together
    df = pd.concat([df, ent_scores], axis=1)
    # Drop unneeded columns
    df.drop(columns=['token_acc', 'token_p', 'token_r', 'token_f', 'ents_per_type'], inplace=True)
    
    return df

In [75]:
dataframes = []
for file in files:
    df = make_df(file)
    dataframes.append(df)

In [77]:
results_df = pd.concat(dataframes, axis=0)

In [78]:
results_df.head()

,ents_p,ents_r,ents_f,speed,model,H_SKILL.p,H_SKILL.r,H_SKILL.f,S_SKILL.p,S_SKILL.r,S_SKILL.f,LANG_FRAM.p,LANG_FRAM.r,LANG_FRAM.f,REQUIREMENT.p,REQUIREMENT.r,REQUIREMENT.f
0,0.371991,0.266876,0.310786,37761.973299,charembed_bilstm,0.122995,0.112745,0.117647,0.558824,0.233129,0.329004,0.589744,0.414414,0.486772,0.369565,0.354167,0.361702
0,0.515464,0.313972,0.390244,29373.816001,charembed_bilstm_vec,0.273684,0.127451,0.173913,0.454545,0.245399,0.318725,0.679487,0.477477,0.560847,0.571429,0.583333,0.577320
0,0.535714,0.32967,0.408163,28638.609691,charembed_maxout,0.284483,0.161765,0.206250,0.514019,0.337423,0.407407,0.725191,0.427928,0.538244,0.710526,0.562500,0.627907
0,0.50358,0.33124,0.399621,24591.112316,charembed_maxout_vec,0.259259,0.068627,0.108527,0.378788,0.306748,0.338983,0.644444,0.522523,0.577114,0.584906,0.645833,0.613861
0,0.561605,0.307692,0.397566,27055.712442,charembed_mish,0.314286,0.161765,0.213592,0.545455,0.331288,0.412214,0.763636,0.378378,0.506024,0.714286,0.520833,0.602410
0,0.491765,0.3281,0.393597,25553.423109,charembed_mish_vec,0.223140,0.132353,0.166154,0.457143,0.294479,0.358209,0.636364,0.472973,0.542636,0.852941,0.604167,0.707317
0,0.51105,0.290424,0.37037,37741.592798,multihash_bilstm,0.217822,0.107843,0.144262,0.494505,0.276074,0.354331,0.698529,0.427928,0.530726,0.676471,0.479167,0.560976
0,0.503686,0.321821,0.39272,36884.00505,multihash_bilstm_vec,0.224719,0.098039,0.136519,0.510000,0.312883,0.387833,0.603352,0.486486,0.538653,0.666667,0.541667,0.597701
0,0.530504,0.313972,0.394477,19277.556476,multihash_maxout,0.251969,0.156863,0.193353,0.548387,0.312883,0.398438,0.744186,0.432432,0.547009,0.750000,0.437500,0.552632
0,0.561097,0.353218,0.433526,26372.005712,multihash_maxout_vec,0.336134,0.196078,0.247678,0.471910,0.257669,0.333333,0.713415,0.527027,0.606218,0.896552,0.541667,0.675325


In [119]:
# Get in long format for some plots
long_df = results_df.melt(id_vars='model')

In [120]:
long_df.head()

,model,variable,value
0,charembed_bilstm,ents_p,0.371991
1,charembed_bilstm_vec,ents_p,0.515464
2,charembed_maxout,ents_p,0.535714
3,charembed_maxout_vec,ents_p,0.50358
4,charembed_mish,ents_p,0.561605


In [168]:
def plot_overall_results(df, sort=None):
    # Filter for precision, recall and f-score
    df = df.loc[df['variable'].isin(['ents_p', 'ents_r', 'ents_f'])]
    # Make barplot
    fig = px.bar(df, x='model', y='value', color='variable', barmode='group', labels={'value':'Score', 'model':'Model', 'variable':'Metric'})
    # Sort by given variable
    if sort is not None:
        sorted_df = df.loc[df['variable'] == sort].sort_values(by='value')
        sorted_array = sorted_df['model'].to_list()
        fig.update_layout(xaxis={'categoryorder':'array', 'categoryarray':sorted_array})
    # Update legend names
    fig.data[0].name = 'Precision'
    fig.data[1].name = 'Recall'
    fig.data[2].name = 'F-score'
    
    fig.show()

In [169]:
plot_overall_results(long_df, sort='ents_f')